<h1><span style="color:red">Base de Datos Relacionales para una Red Social</span></h1>

<h2><span style="color:blue">1. Descripción del Proyecto</span></h2>
<p>En este proyecto crearemos una serie de clases de python para crear una simulación de base de datos para una red social basada en texto que posea las siguientes características:
<ul>
 <li>Que solo los administradores puedan realizar queries no preestablecidas o crear/eliminar las tablas de la base de datos utilizando una contraseña.
 <li>Almacene la información personal de los usuarios al momento de estos registrarse en la red social.
 <li>Genere un registro que permita relacionar seguidores con seguidos dentro de la red social.
 <li>Almacene las públicaciones en formato de texto de los usuarios y les asigne un registro único que relacione        publicación y publicador.
 <li>Lleve una lista de que usuarios hicieron like a que publicaciones y se puedan ver quienes son los usuarios con mayor número de seguidores.
 <li>Que las bases de datos actualicen su información en caso algun usuario deje de seguir a otro o deje de dar like a alguna publicacion.
 <li>Que se puedan consultar de manera pública las publicaciones de un usuario junto con sus respectivos likes.
 <li>La base de datos debe estar almacenada en el ordenador local.
</ul>
Para todo lo anterior utilizaremos el módulo de <a HREF="https://docs.python.org/es/3/library/sqlite3.html">sqlite3</a> que nos permitira crear bases de datos relacionales utilizando el lenguaje SQL mediante la librería <a HREF="https://www.sqlite.org/index.html">SQLite</a>, tambien trabajaremos con Pandas; haciendo asi que podamos trabajar dentro de un notebook de Jupyter Lab. 
</p>

In [1]:
# %load_ext sql <-- Este se usaria si no tenemos dicha extensión instalada.
import sqlite3
import pandas as pd

<h2><span style="color:blue">2. Creando las Bases de Datos</span></h2>
<p>Para cumplir con las condiciones que se nos piden, primero debemos crear la estructura de la Base de Datos, para ello sera necesario crear 4 tablas con las siguientes caracteristicas:
<lu>
<li><b>Tabla de Usuarios:</b> Que almacenara todas la información del usuario para que se registre, a saber: Su Seudónimo (que debe ser único), nombre (primero y segundo), teléfono y correo electrónico, además de llevar registro de cuando se creo la cuenta. Por razones obvias esta tabla no dependera de ninguna otra.
<li><b>Tabla de Seguidores:</b> Que almacene el quien sigue a quien y donde evidentemente no se puede seguir a uno mismo o lo que es igual debe poseer: la identificación del usuario seguidor y la identificación del usuario seguido. Por lo anterior esta tabla solo dependera de la Tabla de Usuarios.
<li><b>Tabla de Publicaciones:</b> Esta tabla debe almacenar: la identificación de la publicación, el texto de la publicación, los likes de la publicación y por supuesto la fecha en la que se hizo la publicación. De esta manera, se relacionara la tabla con la Tabla de Usuarios y la Tabla de Likes.
<li><b>Tabla de likes:</b> Esta tabla debe contener un registro de quien realizo el like y a cual publicación la hizo. Se relacionara entonces con la Tabla de Usuarios y la Tabla de Publicaciones. 
</lu>
Con todo lo anterior tendremos que la estructura de nuestra base de datos es de la siguiente manera:
</p>

![imagen](https://raw.githubusercontent.com/juanetoboneto/Portafolio/main/DataSets/bases_de_datos_relacionales_i/Diagrama_de_base_de_datos_i.jpg)

<p>
Teniendo en cuenta lo que se nos pide, debemos limitar las <EM>queries</EM> que realizamos a la base de datos para que únicamente puedan ser ejecutadas por los administradores, lo cual puede ser algo tedioso ya que se deben cubrir bastantes posibilidades. A pesar de ello, servirá para optimizar las demás clases que necesitamos crear. Todo esto, junto con la creación y eliminación de las tablas, lo llevaremos a cabo dentro de una clase que llamaremos <I>Administrador</I>, como sigue:"
</p>

In [2]:
#Clase con las funcionalidades de Administrador :
class administrador:
 
 # Empecemos con las funciones de consultas para la Data Base:

 #Función para verificar la contraseña de administrador:    
 contraseña = "contraseñaxd" 
 def verificar_contraseña(contraseña_ingresada):
    return contraseña_ingresada == administrador.contraseña

 #Funcion para realizar las querys de sqlite3 (sin objetos instanciados):
 def query(contraseña_ingresada, peticion):
    if administrador.verificar_contraseña(contraseña_ingresada):
        path = 'C:\\Users\\User\\Desktop\\Portafolio\\DataSets\\bases_de_datos_relacionales_i\\red_social_database.db'
        conexion = sqlite3.connect(path)
        cursor = conexion.cursor()
        cursor.execute(peticion)
        conexion.close()
    else:
        pass
    
 #Funcion de Query con commit (sin objetos instanciados):
 def query_commit(contraseña_ingresada, peticion):
    if administrador.verificar_contraseña(contraseña_ingresada):
        path = 'C:\\Users\\User\\Desktop\\Portafolio\\DataSets\\bases_de_datos_relacionales_i\\red_social_database.db'
        conexion = sqlite3.connect(path)
        cursor = conexion.cursor()
        cursor.execute(peticion)
        conexion.commit()
        conexion.close()
    else:
        pass
    
 # Query con objetos instanciados:
 def query_instanciado(contraseña_ingresada, peticion, objetos):
    if administrador.verificar_contraseña(contraseña_ingresada):
        path = 'C:\\Users\\User\\Desktop\\Portafolio\\DataSets\\bases_de_datos_relacionales_i\\red_social_database.db'
        conexion = sqlite3.connect(path)
        cursor = conexion.cursor()
        cursor.execute(peticion, objetos)
        conexion.close()
    else:
        pass

 # Query de commit con objetos instanciados:
 def query_instanciado_commit(contraseña_ingresada, peticion, objetos):
    if administrador.verificar_contraseña(contraseña_ingresada):
        path = 'C:\\Users\\User\\Desktop\\Portafolio\\DataSets\\bases_de_datos_relacionales_i\\red_social_database.db'
        conexion = sqlite3.connect(path)
        cursor = conexion.cursor()
        cursor.execute(peticion, objetos)
        conexion.commit()
        conexion.close()
    else:
        pass

 # Un Query instanciado con petición:
 def query_busqueda(contraseña_ingresada, peticion, objetos):
    if administrador.verificar_contraseña(contraseña_ingresada):
        path = 'C:\\Users\\User\\Desktop\\Portafolio\\DataSets\\bases_de_datos_relacionales_i\\red_social_database.db'
        conexion = sqlite3.connect(path)
        cursor = conexion.cursor()
        cursor.execute(peticion, objetos)
        resultado = str(cursor.fetchone()[0])
        conexion.close()
        return resultado
    else:
        pass
    
  # Query de busqueda múltiple:
 def query_busqueda_multiple(contraseña_ingresada, peticion, objetos):
    if administrador.verificar_contraseña(contraseña_ingresada):
        path='C:\\Users\\User\\Desktop\\Portafolio\\DataSets\\bases_de_datos_relacionales_i\\red_social_database.db'
        conexion = sqlite3.connect(path)
        cursor = conexion.cursor()
        cursor.execute(peticion, objetos)
        resultado = cursor.fetchall()
        conexion.close()
        return resultado
    else:
        pass
    
 # Query de busqueda múltiple sin objetos:
 def query_BMSO(contraseña_ingresada, peticion):
    if administrador.verificar_contraseña(contraseña_ingresada):
        path='C:\\Users\\User\\Desktop\\Portafolio\\DataSets\\bases_de_datos_relacionales_i\\red_social_database.db'
        conexion = sqlite3.connect(path)
        cursor = conexion.cursor()
        cursor.execute(peticion)
        resultado = cursor.fetchall()
        conexion.close()
        return resultado
    else:
        pass
    
# Con ello podremos empezar las funciones de administrador para las tablas de la Data Base:
    
 #Funcionalidades de administrador para la tabla de usuarios:

 def crear_tabla_usuarios(contraseña_ingresada):
    if administrador.verificar_contraseña(contraseña_ingresada):    
     try:
      administrador.query(contraseña_ingresada, """CREATE TABLE IF NOT EXISTS tabla_usuarios (
                        identificacion_usuario INTEGER PRIMARY KEY,
                        seudonimo_usuario VARCHAR(20) NOT NULL UNIQUE,
                        correo_electronico VARCHAR(40) NOT NULL UNIQUE,
                        primer_nombre VARCHAR(80) NOT NULL,
                        segundo_nombre VARCHAR(80) NOT NULL,
                        telefono VARCHAR(12) UNIQUE,
                        creado_el TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP )""")
      print("Se pudo crear la Tabla de Usuarios correctamente.")   
     except Exception as e:
            print(f"No se pudo crear la tabla de usuarios. Error: {e}")   
    else:
        print("Contraseña incorrecta")
    
 def eliminar_tabla_usuarios(contraseña_ingresada):
    if administrador.verificar_contraseña(contraseña_ingresada):
     try:
      administrador.query(contraseña_ingresada,"""DROP TABLE IF EXISTS tabla_usuarios""")
      print("Se pudo eliminar la Tabla de Usuarios correctamente.") 
     except Exception as e:
            print(f"No se pudo eliminar la tabla de usuarios. Error: {e}")
    else:
        print("Contraseña incorrecta")
    
 #Funcionalidades de administrador para la tabla de seguidores:
 #Nota: esta tabla tiene la condición de que nadie puede seguirse a sí mismo
 def crear_tabla_seguidores(contraseña_ingresada):
    if administrador.verificar_contraseña(contraseña_ingresada):
        try:
         administrador.query(contraseña_ingresada, """CREATE TABLE IF NOT EXISTS tabla_seguidores (
                           identificacion_seguidor INT NOT NULL,
                           identificacion_seguido INT NOT NULL,
                           FOREIGN KEY (identificacion_seguidor) REFERENCES tabla_usuarios(identificacion_usuario),
                           FOREIGN KEY (identificacion_seguido) REFERENCES tabla_usuarios(identificacion_usuario),
                           PRIMARY KEY (identificacion_seguidor, identificacion_seguido) )""")
         print("Se pudo crear la Tabla de Seguidores correctamente.")
        except Exception as e:
            print(f"No se pudo crear la Tabla de de Seguidores. Error: {e}")
    else:
        print("Contraseña incorrecta") 

 def eliminar_tabla_seguidores(contraseña_ingresada):
    if administrador.verificar_contraseña(contraseña_ingresada):
        try:
         administrador.query(contraseña_ingresada, """DROP TABLE IF EXISTS tabla_seguidores""")
         print("Se pudo eliminar la Tabla de Seguidores correctamente.")
        except Exception as e:
            print(f"No se pudo eliminar la Tabla de Seguidores. Error: {e}")
    else:
        print("Contraseña incorrecta")
        
 #Funcionalidades de administrador para la tabla de publicaciones:
 
 def crear_tabla_publicaciones(contraseña_ingresada):
    if administrador.verificar_contraseña(contraseña_ingresada):
        try:
         administrador.query(contraseña_ingresada, """CREATE TABLE IF NOT EXISTS tabla_publicaciones (
                           identificacion_publicacion INTEGER PRIMARY KEY,
                           identificacion_usuario INT NOT NULL,
                           texto_publicacion VARCHAR(12000) NOT NULL,
                           numero_likes INT DEFAULT 0,
                           creado_en TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
                           FOREIGN KEY (identificacion_usuario) REFERENCES tabla_usuarios(identificacion_usuario) )""")
         print("Se pudo crear la Tabla de Publicaciones correctamente.")
        except Exception as e:
            print(f"No se pudo crear la Tabla de Publicaciones. Error:{e}")
    else:
        print("Contraseña incorrecta")
            
 def eliminar_tabla_publicaciones(contraseña_ingresada):
    if administrador.verificar_contraseña(contraseña_ingresada):
        try:
         administrador.query(contraseña_ingresada, """DROP TABLE IF EXISTS tabla_publicaciones""")
         print("Se pudo eliminar la Tabla de Publicaciones correctamente.")
        except Exception as e:
            print(f"No se pudo eliminar la Tabla de Publicaciones. Error:{e}")
    else:
        print("Contraseña incorrecta")
            
 #Funcionalidades de administrador para la tabla de "likes":

 def crear_tabla_likes(contraseña_ingresada):
    if administrador.verificar_contraseña(contraseña_ingresada):
        try:
         administrador.query(contraseña_ingresada, """CREATE TABLE IF NOT EXISTS tabla_likes (
                        identificacion_usuario INT NOT NULL,
                        identificacion_publicacion INT NOT NULL,
                        FOREIGN KEY (identificacion_usuario) REFERENCES tabla_usuarios(identificacion_usuario),
                        FOREIGN KEY (identificacion_publicacion) REFERENCES tabla_publicaciones(identificacion_publicacion),
                        PRIMARY KEY (identificacion_usuario, identificacion_publicacion))""")
         print("Se pudo crear la Tabla de Likes correctamente.")
        except Exception as e:
            print(f"No se pudo crear la Tabla de Likes. Error:{e}")
    else:
        print("Contraseña incorrecta")

 def eliminar_tabla_likes(contraseña_ingresada):
    if administrador.verificar_contraseña(contraseña_ingresada):
        try:
         administrador.query(contraseña_ingresada, """DROP TABLE IF EXISTS tabla_likes""")
         print("Se pudo eliminar la Tabla de Likes correctamente.")
        except Exception as e:
            print(f"No se pudo eliminar la Tabla de Likes. Error:{e}")
    else:
        print("Contraseña incorrecta")
        
 # Operaciones con multiples tablas:

 def crear_todas_las_tablas(contraseña_ingresada):
   administrador.crear_tabla_usuarios(contraseña_ingresada)
   administrador.crear_tabla_seguidores(contraseña_ingresada)
   administrador.crear_tabla_publicaciones(contraseña_ingresada)
   administrador.crear_tabla_likes(contraseña_ingresada)

 def eliminar_todas_las_tablas(contraseña_ingresada):
  administrador.eliminar_tabla_usuarios(contraseña_ingresada)
  administrador.eliminar_tabla_seguidores(contraseña_ingresada)
  administrador.eliminar_tabla_publicaciones(contraseña_ingresada)
  administrador.eliminar_tabla_likes(contraseña_ingresada)

<h2><span style="color:blue">3. Creando las Funcionalidades de Usuario</span></h2>

<p>
La siguiente clase debe poseer todas las funciones que solo estan disponibles para las personas registradas en la base de datos, esto quiere decir que necesariamente estan registrados en la red social, o lo que es igual existen valores de ellos en la Tabla de Usuarios. Por las condiciones que se nos fueron dadas deben existir al menos las siguientes funciones:
<lu>
<li>Almacenar los datos de un usuario al registrarse en la red social y que, si este lo desea, eliminar su cuenta y la información que dio.
<li>Despues de registrado, ser capaz de seguir o dejar de seguir otras cuentas.
<li>Poder publicar comentarios de texto y ser capaz de eliminarlos dado el caso.
<li>Poder dar like o anular el like a comentarios propios o de otros usuarios.
</lu>
<br>
Para todas estas funcionalidades crearemos la clase <EM>Usuarios</EM>:
</p>

In [3]:
# Creemos una clase que sintetize todas las funcionalidades para el usuario:
class usuario:
 
 # Funcionalidades de Usuario para la tabla de usuarios:
    
 # Utilizaremos el instanciamiento de un usuario en la clase como su registro en la red social: 
 def __init__(self,seudonimo_usuario, correo_electronico, primer_nombre, segundo_nombre, telefono):
    
    self.seudonimo_usuario = seudonimo_usuario
    self.correo_electronico = correo_electronico
    self.primer_nombre = primer_nombre
    self.segundo_nombre = segundo_nombre
    self.telefono = telefono

    administrador.query_instanciado_commit("contraseñaxd", """INSERT INTO tabla_usuarios 
                   (seudonimo_usuario, correo_electronico, primer_nombre, segundo_nombre, telefono) VALUES(?,?,?,?,?)""", 
                   (self.seudonimo_usuario, self.correo_electronico, self.primer_nombre, self.segundo_nombre, self.telefono))
    
    print(f"Su cuenta {seudonimo_usuario} ha sido creada correctamente.")
    
    
 def eliminar_usuario(usuario):
  try:
    administrador.query_instanciado_commit("contraseñaxd", """DELETE  FROM tabla_usuarios 
                                           WHERE seudonimo_usuario = ?""", (usuario.seudonimo_usuario,))
    print(f"La cuenta {usuario.seudonimo_usuario} fue eliminada correctamente.")
  except Exception as e:
    print(f"No se pudo eliminar la cuenta. Error:{e}")

 
 # Funcionalidades de la tabla de seguidores:

 # Estas funcionalidades son algo más complicadas debido a que necesitaremos realizar queris por orden a la tabla de usuarios
 # para poder luego guardarlas en la tabla de seguidores:

 def seguir_usuario(usuario, seguir):
  try:
   # realizando la query del ID del seguidor:     
   x_1 = administrador.query_busqueda("contraseñaxd", """SELECT identificacion_usuario 
                    FROM tabla_usuarios WHERE seudonimo_usuario=?""", (usuario.seudonimo_usuario, ))
   # realizando la query del ID del seguido:
   x_2 = administrador.query_busqueda("contraseñaxd", """SELECT identificacion_usuario 
                    FROM tabla_usuarios WHERE seudonimo_usuario=?""", (seguir, ))
   # guardando dicha relacion en la tabla de seguidores:
   administrador.query_instanciado_commit("contraseñaxd", """INSERT INTO tabla_seguidores
                                          (identificacion_seguidor, identificacion_seguido) VALUES(?,?)""", (x_1, x_2))
   print(f"{usuario.seudonimo_usuario} ahora sigue a {seguir}")
  except Exception as e:
    print("Tu cuenta o la de la persona que intentas seguir no existen.")
    
 def dejar_de_seguir_usuario(usuario, dejar_de_seguir):
    try:
     # realizando la query del ID del seguidor:
     x_1 = administrador.query_busqueda("contraseñaxd", """SELECT identificacion_usuario 
                                  FROM tabla_usuarios WHERE seudonimo_usuario=?""", (usuario.seudonimo_usuario, ))
     # realizando la query del ID del seguido:
     x_2 = administrador.query_busqueda("contraseñaxd", """SELECT identificacion_usuario 
                      FROM tabla_usuarios WHERE seudonimo_usuario=?""", (dejar_de_seguir, ))
     # eliminando dicha relación de la tabla de seguidores:
     administrador.query_instanciado_commit("contraseñaxd", """DELETE FROM tabla_seguidores
                                            WHERE (identificacion_seguidor, identificacion_seguido) = (?, ?)""", (x_1, x_2))
     print(f"{usuario.seudonimo_usuario} dejo de seguir a {dejar_de_seguir}")
    except Exception as e:
        print(f"No se pudo anular el seguido: Error {e}")
        
 # Funcionalidades de usuario para la Tabla de Publicaciones:

 # Podriamos utilizar una condición "if" para asegurarnos que la entrada de las publicaciones sea necesariamente
 # un string, sin embargo esto extenderia innecesariamente el código, solo para que a efectos prácticos se tenga 
 # el mismo resultado que introducir una variable "normal".
    
 def publicar_comentario(usuario, publicacion):
    try:
     id_usuario = administrador.query_busqueda("contraseñaxd", """SELECT identificacion_usuario 
                      FROM tabla_usuarios WHERE seudonimo_usuario=?""", (usuario.seudonimo_usuario, ))
     administrador.query_instanciado_commit("contraseñaxd", """INSERT INTO tabla_publicaciones 
                            (identificacion_usuario, texto_publicacion) VALUES(?,?)""", (id_usuario, publicacion))
     print("Su comentario a sido publicado con éxito.")
    except Exception as e:
     print(f"Tu comentario no pudo ser publicado. Error:{e}")
  
 # Para la funcion de eliminar publicacion necesitaremos realizar 2 querys, una que nos de el ID del usuario y
 # otra que nos permita eliminar su comentario:
 def eliminar_publicacion(usuario, publicacion):
  try:
   id_usuario = administrador.query_busqueda("contraseñaxd", """SELECT identificacion_usuario FROM tabla_usuarios 
                                WHERE seudonimo_usuario=?""", (usuario.seudonimo_usuario, ))
   administrador.query_instanciado_commit("contraseñaxd", """DELETE FROM tabla_publicaciones 
                                     WHERE (identificacion_usuario, texto_publicacion) = (?,?)""",(id_usuario, publicacion))
   print("Su publicación a sido eliminado con éxito.")
  except Exception as e:
   print(f"Tu publicación no pudo ser eliminada. Error:{e}")
    
    
 # Funcionalidades de usuario para la Tabla de Likes:

 def dar_like(usuario, publicador, publicacion):
  try:
   id_publicador = administrador.query_busqueda("contraseñaxd", """SELECT identificacion_usuario FROM tabla_usuarios 
                    WHERE seudonimo_usuario=?""", (publicador, ))
   id_publicacion = administrador.query_busqueda("contraseñaxd", """SELECT identificacion_publicacion FROM tabla_publicaciones 
                    WHERE (identificacion_usuario, texto_publicacion)=(?,?)""", (id_publicador, publicacion))
   administrador.query_instanciado_commit("contraseñaxd", """INSERT INTO tabla_likes 
                    (identificacion_usuario, identificacion_publicacion) VALUES(?,?)""", (id_publicador, id_publicacion))
   print(f"A {usuario.seudonimo_usuario} le a gustado {publicacion}")
  except Exception as e:
   print(f"No se pudo dar like. Error:{e}") 

 def anular_like(usuario, publicador, publicacion):
  try:
   id_publicador = administrador.query_busqueda("contraseñaxd", """SELECT identificacion_usuario FROM tabla_usuarios 
                    WHERE seudonimo_usuario=?""", (publicador, ))
   id_publicacion = administrador.query_busqueda("contraseñaxd", """SELECT identificacion_publicacion FROM tabla_publicaciones 
                    WHERE (identificacion_usuario, texto_publicacion)=(?,?)""", (id_publicador, publicacion))
   administrador.query_instanciado_commit("contraseñaxd", """DELETE FROM tabla_likes 
                    WHERE (identificacion_usuario, identificacion_publicacion)=(?,?)""", (id_publicador, id_publicacion))
  except Exception as e:
    print(f"No se pudo anular tu like. Error: {e}")

<h2><span style="color:blue">4. Funcionalidades Públicas</span></h2>

<p> En nuestra base de datos, además de realizar cambios como administradores y consultas como usuarios, también es necesario que podamos extraer cierta información de manera totalmente pública. Para ello, crearemos una clase diferenciada, la cual no deberá instanciar objetos ni utilizar contraseñas para obtener datos y que debe tener las siguientes funcionalidades:
<ul>
  <li>Que cualquiera en la red pueda consultar el nombre de un usuario en la red social.
  <li>Que cualquiera pueda encontrar la identificación o registro de una publicación dentro de la red social.
  <li>Se pueda ver una lista de las publicaciones de cualquier persona junto con el número de likes de cada una.
  <li>Cualquiera pueda observar quiénes son los 3 usuarios más seguidos en orden descendente y su respectivo número de seguidores. 
</ul>
Para todas estas funcionalidades que faltan, creamos la clase <EM>Público</EM>:
</p>

In [4]:
class publico:

 # Funcion para buscar usuarios:
 def buscar_usuario(usuario_buscado):
  try:
    usuario_encontrado = administrador.query_busqueda_multiple("contraseñaxd",
                          """SELECT primer_nombre, segundo_nombre FROM tabla_usuarios 
                             WHERE seudonimo_usuario=?""",(usuario_buscado,))
    print(f"El nombre del usuario al que buscas es: {usuario_encontrado[0][0]} {usuario_encontrado[0][1]}.")
  except Exception as e:
    print(f"No se pudo encontrar al Usuario. Error:{e}")
    
 # Buscar la identificacion de una publicacion   
 def buscar_registro_publicacion(usuario_publicador, texto_publicacion):
  try:
    id_usuario = administrador.query_busqueda("contraseñaxd", """SELECT identificacion_usuario 
                      FROM tabla_usuarios WHERE seudonimo_usuario=?""", (usuario_publicador, ))
    id_publicaciones = administrador.query_busqueda("contraseñaxd", """SELECT identificacion_publicacion 
       FROM tabla_publicaciones WHERE (identificacion_usuario, texto_publicacion) = (?,?)""", (id_usuario, texto_publicacion))
    print(f"El número de registro de la publicación del usuario {usuario_publicador} es {id_publicaciones}.")
  except Exception as e:
    print(f"Ocurrio un problema durante la busqueda. Error:{e}")

 # Generar una lista de comentarios junto con likes:
 def lista_publicaciones(usuario_publicador):
  try:
   id_usuario = administrador.query_busqueda("contraseñaxd", """SELECT identificacion_usuario 
                      FROM tabla_usuarios WHERE seudonimo_usuario=?""", (usuario_publicador, ))
   informacion = administrador.query_busqueda_multiple("contraseñaxd",
                    """SELECT texto_publicacion, COUNT(*) AS numero_likes FROM tabla_publicaciones
                       WHERE identificacion_usuario =?""", (id_usuario,))
   informacion = pd.DataFrame(informacion, columns=["Comentario", "Número de Likes"])
   print(informacion)
  except Exception as e:
    print(f"Ocurrio algun fallo en la busqueda. Error:{e}")
    
 # Top 3 los usuarios más seguidos:
 def top_seguidos():
  try:
    lista = administrador.query_BMSO("contraseñaxd", 
             """SELECT seudonimo_usuario, COUNT(identificacion_seguidor) FROM tabla_seguidores 
                JOIN tabla_usuarios ON (tabla_usuarios.identificacion_usuario = tabla_seguidores.identificacion_seguido)
                GROUP BY identificacion_seguido ORDER BY COUNT(identificacion_seguidor) DESC LIMIT 3""")
    lista = pd.DataFrame(lista, columns=["Usuario", "Número de seguidores"])
    print(lista)
  except Exception as e:
    print(f"No se pudo cargar la lista. Error{e}")

<h2><span style="color:blue">5. Probando nuestras Bases de Datos</span></h2>
<p>Por último veamos como se comportan las funcionalidades de las diferentes clases el momento de interactuar con la base de dato relacional de nuestra red social, esto para comprobar de primera mano que satisface las condiciones que se nos pidieron al principio.
<BR>
Empecemos intentando crear las tablas sin la contraseña de administrador:


In [5]:
administrador.crear_tabla_usuarios("No la contraseña correcta")

Contraseña incorrecta


<p>Ahora veamos como se pueden eliminar y crear toda la estructura de bases de datos a la vez, siendo un administrador con la contraseña correcta:</p>

In [6]:
administrador.crear_todas_las_tablas("contraseñaxd")

Se pudo crear la Tabla de Usuarios correctamente.
Se pudo crear la Tabla de Seguidores correctamente.
Se pudo crear la Tabla de Publicaciones correctamente.
Se pudo crear la Tabla de Likes correctamente.


<p>Tambien, creemos un ejemplo de 3 Usuarios que se instancian como objetos de dicha clase. Registrandose en la base de datos de la red social:</p>

In [7]:
juan = usuario("juan", "juanes@gmail.com", "juan", "esteban", "3014415299")
jose = usuario("jose", "josefo@gmail.com", "jose", "manuel", "3113739070")
pepe = usuario("pepe", "pepe@gmail.com", "pepe", "gomez", "3122520503")

Su cuenta juan ha sido creada correctamente.
Su cuenta jose ha sido creada correctamente.
Su cuenta pepe ha sido creada correctamente.


<p>Imaginemos a alguien que no posee una cuenta en la red social y busca a un usuario en específico:</p>

In [8]:
publico.buscar_usuario("juan")

El nombre del usuario al que buscas es: juan esteban.


<p>Ahora veamos que esa misma persona intenta eliminar la cuenta de ese usuario registrado, este ejemplo tambien serviría para alguien que SI esta registrado y que intenta eliminar la cuenta de otro usuario:</p>

In [9]:
alguien.eliminar_usuario("juan")

NameError: name 'alguien' is not defined

<p>Veamos en cambio, como los usuarios registrados empiezan a seguirse o dejar de seguirse entre ellos:</p>

In [10]:
juan.seguir_usuario("jose")
juan.dejar_de_seguir_usuario("jose")

jose.seguir_usuario("juan")
jose.seguir_usuario("pepe")

pepe.seguir_usuario("juan")
pepe.seguir_usuario("jose")

juan ahora sigue a jose
juan dejo de seguir a jose
jose ahora sigue a juan
jose ahora sigue a pepe
pepe ahora sigue a juan
pepe ahora sigue a jose


<p>Una vez ya establecidas las relaciones entre seguidores y seguidos, veamos (desde la perspectiva de alguien de afuera) como se realiza una consulta de la lista de los más seguidos:</p>

In [11]:
publico.top_seguidos()

  Usuario  Número de seguidores
0    juan                     2
1    pepe                     1
2    jose                     1


<p>Siguiendo con el ejemplo anterior veamos como Juan realiza la primera publicacion de texto dentro de la red y como Jose le da like al mismo:</p>

In [12]:
juan.publicar_comentario("Hola gente, este es mi primer comentario siganme en esta red social plz")
jose.dar_like("juan", "Hola gente, este es mi primer comentario siganme en esta red social plz")

Su comentario a sido publicado con éxito.
A jose le a gustado Hola gente, este es mi primer comentario siganme en esta red social plz


<p>Veamos como una persona de fuera de la red busca el código de registro que tiene el comentario de Juan:</p>

In [13]:
publico.buscar_registro_publicacion("juan", "Hola gente, este es mi primer comentario siganme en esta red social plz")

El número de registro de la publicación del usuario juan es 1.


<p>Ahora veamos como alguien al que le interesa las públicaciones de Juan, pero no esta registrado en la red, puede consultar dichas publicaciones y los "likes" respectivos de cada una:</p>

In [14]:
publico.lista_publicaciones("juan")

                                          Comentario  Número de Likes
0  Hola gente, este es mi primer comentario sigan...                1


<p>Veamos el caso en el que Juan se arrepiente de la publicación que realizo y decide eliminarla:</p>

In [15]:
juan.eliminar_publicacion("Hola gente, este es mi primer comentario siganme en esta red social plz")

Su publicación a sido eliminado con éxito.


<p>Podriamos eliminar toda la estructura de datos que hicimos. Sin embargo, por razones didácticas prefiero no hacerlo.</p>